# Pre-processing and Feature Engineeing

In [1]:
import sys
from pathlib import Path

project_root = Path().resolve().parent

sys.path.append(str(project_root))

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from config.features import get_drop_features

In [3]:
train_identity = pd.read_csv('../data/train_identity.csv')
identity_columns = train_identity.columns.difference(['TransactionID'])

print("Identity dataset load sucessfully!")

Identity dataset load sucessfully!


In [4]:
merged_chunks = []
transaction_chunks = pd.read_csv('../data/train_transaction.csv', chunksize=10 ** 5)

for chunk in transaction_chunks:
    new_chunk = chunk.merge(train_identity, on='TransactionID', how='left')
    merged_chunks.append(new_chunk)

train = pd.concat(merged_chunks)
train.shape

(590540, 434)

In [5]:
train['hasIdentity'] = (~train[identity_columns].isna().all(axis=1)).astype(int)

In [6]:
train = train.drop(columns=get_drop_features(train.columns))
train.head(5)

Previous features: 435
Relevant features: 291
Final training set features: 283


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_30,id_31,id_32,id_33,id_36,id_37,id_38,DeviceType,DeviceInfo,hasIdentity
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,Android 7.0,samsung browser 6.2,32.0,2220x1080,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,1


Make sure to delete the unnecessary variables from memory to relieve some of it, so it will not bother us throughout the script. Also, let's save this merged dataset to optimize future reading operations.

In [7]:
del train_identity, transaction_chunks

### Feature Selection

In [8]:
many_null_cols = [col for col in train.columns if (train[col].isnull().sum() / train.shape[0]) >= 0.8]
len(many_null_cols)

45

In [9]:
train = train.drop(columns=many_null_cols)
train.shape

(590540, 238)

### Feature Engineering

In [10]:
amount = 'TransactionAmt'
selected_cols = ['card1', 'card4', 'P_emaildomain', 'R_emaildomain', 'addr1']

for col in selected_cols:
    train[f'amount_mean_{col}'] = train[amount] / train.groupby([col])[amount].transform('mean')
    train[f'amount_std_{col}'] = train[amount] / train.groupby([col])[amount].transform('std')

train['TransactionAmtLog'] = np.log(train[amount])
train['TransactionAmtCents'] = (train[amount] - np.floor(train[amount])).astype(np.float64)

#### Time Delta Feature

TransactionDT: timedelta from a given reference datetime (not an actual timestamp)

In [11]:
start_date = datetime.strptime('2022-01-01', '%Y-%m-%d')

train['Date'] = train['TransactionDT'].apply(lambda dt: start_date + timedelta(seconds=dt))

train['Weekday'] = train['Date'].dt.dayofweek
train['Day'] = train['Date'].dt.day
train['Hour'] = train['Date'].dt.hour

train = train.drop(columns=['Date'])

In [12]:
train[['TransactionDT', 'Weekday', 'Day', 'Hour']].sample(10)

,TransactionDT,Weekday,Day,Hour
57787,6192344,6,13,16
77055,12347530,0,23,21
53560,3180565,6,6,19
48970,3078953,5,5,15
11290,7753066,3,31,17
50182,14521266,5,18,1
25781,10774027,3,5,16
82294,15548783,2,29,23
63930,1451159,0,17,19
80611,12497445,2,25,15


In [13]:
train.to_parquet('../data/train.parquet', index=False)